## TFM: versión LDA  

En esta versión del documento, vamos a tratar de analizar los tweets de la RAE de manera cuantitativa y clasificativa haciendo uso de la herramienta LDA y la librería **PANDAS**  

### Loading data
---
Primero debemos importar los tweets de un documento *excel* o *csv*:
``` python 

    # Importing modules
    import pandas as pd
    import os
    
    # Read data into papers
    papers = pd.read_csv('../../tweets.csv') #ruta
    
    # Print head
    papers.head()

    ```

In [15]:
#poner código válido aquí:
import pandas as pd
import os 

papers = pd.read_csv('tuits.csv')

papers.head()

,_id,text,userid,screen_name,created_at,source,hashtags,symbols,user_mentions,lang,...,coordinates,reply,in_reply_to_user_id_str,in_reply_to_status_id_str,in_reply_to_screen_name,quote,quoted_status_id_str,RT,RT_source,nRTin
0,1334077182629601286,@sare621107 #RAEconsultas Lo que indica el «DL...,350411337,RAEinforma,2020-12-02T10:09:20.000Z,"<a href=""https://www.hootsuite.com"" rel=""nofol...","[{""text"":""RAEconsultas"",""indices"":[12,25]}]",[],"[{""screen_name"":""sare621107"",""name"":""Ernesto S...",es,...,NaN,True,3.198385e+08,1.333900e+18,sare621107,False,NaN,False,NaN,0.0
1,1334077668422201344,"@kunaino #RAEconsultas En efecto, debe de trat...",350411337,RAEinforma,2020-12-02T10:11:16.000Z,"<a href=""https://www.hootsuite.com"" rel=""nofol...","[{""text"":""RAEconsultas"",""indices"":[9,22]}]",[],"[{""screen_name"":""kunaino"",""name"":""Kuña"",""id"":4...",es,...,NaN,True,4.916835e+08,1.333912e+18,kunaino,False,NaN,False,NaN,0.0
2,1334078266215452672,"@so3ndok #RAEconsultas Sí, uno de los usos de ...",350411337,RAEinforma,2020-12-02T10:13:39.000Z,"<a href=""https://www.hootsuite.com"" rel=""nofol...","[{""text"":""RAEconsultas"",""indices"":[9,22]}]",[],"[{""screen_name"":""so3ndok"",""name"":""Tomás Moi"",""...",es,...,NaN,True,1.332657e+18,1.333928e+18,so3ndok,False,NaN,False,NaN,0.0
3,1334078499112542211,"@JorgeBorlandL #RAEconsultas Lo sentimos, pero...",350411337,RAEinforma,2020-12-02T10:14:34.000Z,"<a href=""https://www.hootsuite.com"" rel=""nofol...","[{""text"":""RAEconsultas"",""indices"":[15,28]}]",[],"[{""screen_name"":""JorgeBorlandL"",""name"":""G2 Bol...",es,...,NaN,True,2.252063e+09,1.333952e+18,JorgeBorlandL,False,NaN,False,NaN,0.0
4,1334034386526867458,@jandrote #RAEconsultas La grafía del gerundio...,350411337,RAEinforma,2020-12-02T07:19:17.000Z,"<a href=""https://www.hootsuite.com"" rel=""nofol...","[{""text"":""RAEconsultas"",""indices"":[10,23]}]",[],"[{""screen_name"":""jandrote"",""name"":""josue vega""...",es,...,NaN,True,1.240261e+08,1.333980e+18,jandrote,False,NaN,False,NaN,2.0


### Data cleaning
---
El siguiente paso es limpiar los datos. Eliminaremos las columnas que no nos interesan del documento cargado. Además, se imprimirá la primera fila del documento como título de las otras:
``` python

    # Remove the columns
    papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)
    
    # Print out the first rows of papers
    papers.head()

    ```

In [17]:
#poner código válido aquí:
#eliminar stop words, usuarios mencionados, posibles links y etiquetas
papers = papers.drop(columns= ['text','user_mentions', 'source', 'hashtags'], axis=1).sample(100)
papers.head()

,_id,userid,screen_name,created_at,symbols,lang,quote_count,reply_count,retweet_count,favorite_count,coordinates,reply,in_reply_to_user_id_str,in_reply_to_status_id_str,in_reply_to_screen_name,quote,quoted_status_id_str,RT,RT_source,nRTin
3227,1087318141464649730,350411337,RAEinforma,2019-01-21T11:57:00.000Z,[],es,14.0,3.0,1.0,19.0,NaN,True,827907662.0,1.086453e+18,JJ_MM_99,False,NaN,False,NaN,0.0
2127,1336239168154906625,1263527801883410436,fsiecyl,2020-12-08T09:20:18.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.336023e+18,NaN
2987,1338473399769763842,350411337,RAEinforma,2020-12-14T13:18:20.000Z,[],es,0.0,0.0,0.0,0.0,NaN,True,89814723.0,1.338455e+18,arpestre,False,NaN,False,NaN,0.0
5515,1339537726954213377,1080191192,rqnicolas,2020-12-17T11:47:36.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.339515e+18,NaN
5782,1339572721089429505,120564215,IsaiasCifuentes,2020-12-17T14:06:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.338435e+18,NaN


### Removing
---
En este apartado vamos a limpiar los signos de puntuación que no nos interesen. En este ejemplo se aprecia el borrado de **puntuación**, **mayúsculas** y la **primera fila** del la hoja de datos, pero se pueden añadir, por ejemplo, **_stopwords_**:
``` python

    # Load the regular expression library
    import re
    
    # Remove punctuation
    papers['paper_text_processed'] = \
    papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
    
    # Convert the titles to lowercase
    papers['paper_text_processed'] = \
    papers['paper_text_processed'].map(lambda x: x.lower())
    
    # Print out the first rows of papers
    papers['paper_text_processed'].head()

    ```
    
    

In [16]:
#poner código válido aquí:

### Create the visual *cloud*
---
En este apartado vamos a crear una imagen visual que nos indique a través de su tamaño, cuales son las palabras que mas aparecen en nuestro corpus. Esto no tiene unicamente una función decorativa, sino que podemos :
``` python

    # Import the wordcloud library
    from wordcloud import WordCloud
    
    # Join the different processed titles together.
    long_string = ','.join(list(papers['paper_text_processed'].values))
    
    # Create a WordCloud object
    wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
    
    # Generate a word cloud
    wordcloud.generate(long_string)
    
    # Visualize the word cloud
    wordcloud.to_image()

    ```
    
<img src="cloud.jpg" width="50%" height="50%">


In [17]:
#poner código válido aquí:

### Prepare data for LDA
---
Hay que transformar los datos para que sirvan como input en un **modelo de entrenamiento LDA**. Se tokeniza y se eliminan las _stopwords_, si es que no se ha hecho antes:
``` python

    import gensim
    from gensim.utils import simple_preprocess
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
    
    def sent_to_words(sentences):
        for sentence in sentences:
            # deacc=True removes punctuations
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
            
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) 
                 if word not in stop_words] for doc in texts]
    
    data = papers.paper_text_processed.values.tolist()
    data_words = list(sent_to_words(data))
    
    # remove stop words
    data_words = remove_stopwords(data_words)
    
    print(data_words[:1][0][:30])

    ```

In [18]:
#poner código válido aquí:

### Modelo de entrenamiento LDA
---
En esta parte indicamos a LDA en cuantas categorías se va a clasificar el corpus. En el siguiente ejemplo, vemos que son 10. Como resultado, obtendremos las diez categorias con las palabras que mayor número de apariciones tengan:
``` python

    from pprint import pprint
    
    # number of topics
    num_topics = 10
    
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics)
    
    # Print the Keyword in the 10 topics
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
```

In [19]:
#poner código válido aquí:

### Gráficos
---
En esta seción obtenemos una representación gráfica de los datos:

In [20]:
#poner código válido aquí: